# Approach 2 Enhanced: Cross-Modal Attention Fusion with Tabular Transformer
## Intermediate Fusion with Attention Mechanism + Transformer Tabular Encoder

This notebook trains an enhanced model with:
- **Tabular Transformer**: Self-attention for tabular features (replaces MLP)
- **Cross-Modal Attention**: Image-tabular fusion
- **Attention Visualization**: GradCAM-style visualizations on validation set

### Architecture:
1. **Image Branch**: EfficientNet-B1 → Spatial feature maps (7×7×1280)
2. **Tabular Branch**: **Transformer Encoder** → Context vector (32-dim) [ENHANCED]
3. **Cross-Modal Attention**: Uses tabular context to weight image features
4. **Fusion Network**: Concatenated features → Price prediction
5. **Visualization**: Attention maps on validation samples

In [ ]:
import sys
sys.path.append('../src')

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from config import *
from attention_model import AttentionFusionModel, count_parameters
from dataset import create_dataloaders, prepare_features
from train import train_attention_model, load_checkpoint
from attention_visualizer import AttentionVisualizer, analyze_attention_patterns

print(f"✅ Imports successful")
print(f"Device: {DEVICE}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Load Processed Data

In [ ]:
print("📂 Loading processed data...")

# Load data from Approach 1 preprocessing
train_df = pd.read_csv('../data/train_processed.csv')
val_df = pd.read_csv('../data/val_processed.csv')
test_df = pd.read_csv('../data/test_processed.csv')

# Try to load final_test if it exists
try:
    final_test_df = pd.read_csv('../data/final_test_processed.csv')
except FileNotFoundError:
    print("⚠️  final_test_processed.csv not found, using test_df instead")
    final_test_df = test_df.copy()

print(f"\n📊 Dataset Sizes:")
print(f"  Train:      {len(train_df):>6,} rows")
print(f"  Validation: {len(val_df):>6,} rows")
print(f"  Test:       {len(test_df):>6,} rows")
print(f"  Final Test: {len(final_test_df):>6,} rows")

print(f"\n📊 Images Available:")
for name, df in [('Train', train_df), ('Validation', val_df), ('Test', test_df), ('Final Test', final_test_df)]:
    img_count = df['image_exists'].sum()
    img_pct = 100 * img_count / len(df)
    print(f"  {name:12s} {img_count:>6,} ({img_pct:.1f}%)")

## 2. Prepare Tabular Features

In [ ]:
# Prepare feature columns
feature_columns, train_df = prepare_features(train_df, {
    'NUMERICAL_FEATURES': NUMERICAL_FEATURES,
    'CATEGORICAL_FEATURES': CATEGORICAL_FEATURES,
    'LOG_FEATURES': LOG_FEATURES,
    'EXCLUDE_FEATURES': EXCLUDE_FEATURES
})

# Ensure all dataframes have the same features
for df in [val_df, test_df, final_test_df]:
    for feat in feature_columns:
        if feat not in df.columns:
            df[feat] = 0

print(f"\n📊 Final Feature Set: {len(feature_columns)} features")
print(f"  First 10: {feature_columns[:10]}")

## 3. Create DataLoaders

In [ ]:
# Update config with actual input dimensions
config_dict = {
    'IMAGE_CONFIG': IMAGE_CONFIG,
    'TRANSFORMER_CONFIG': TRANSFORMER_CONFIG,
    'ATTENTION_CONFIG': ATTENTION_CONFIG,
    'FUSION_CONFIG': FUSION_CONFIG,
    'TRAINING_CONFIG': TRAINING_CONFIG,
    'AUGMENTATION_CONFIG': AUGMENTATION_CONFIG,
    'VISUALIZATION_CONFIG': VISUALIZATION_CONFIG,
    'MLFLOW_CONFIG': MLFLOW_CONFIG,
    'NUM_WORKERS': NUM_WORKERS,
    'NUMERICAL_FEATURES': NUMERICAL_FEATURES,
    'CATEGORICAL_FEATURES': CATEGORICAL_FEATURES,
    'LOG_FEATURES': LOG_FEATURES,
    'EXCLUDE_FEATURES': EXCLUDE_FEATURES,
}

# Create dataloaders
dataloaders, stats = create_dataloaders(
    train_df, val_df, test_df,
    feature_columns,
    config_dict
)

## 4. Initialize Model with Tabular Transformer

In [ ]:
print("\n🏗️  Initializing Enhanced Attention Fusion Model with Tabular Transformer...")

model = AttentionFusionModel(
    tabular_input_dim=len(feature_columns),
    config=config_dict
).to(DEVICE)

# Print model architecture details
param_info = count_parameters(model)

print(f"\n📊 Model Parameters:")
print(f"  Total:     {param_info['total']:>12,}")
print(f"  Trainable: {param_info['trainable']:>12,} ({param_info['trainable_pct']:.1f}%)")
print(f"  Frozen:    {param_info['frozen']:>12,}")

if 'components' in param_info:
    print(f"\n📊 Component Breakdown:")
    for comp_name, comp_params in param_info['components'].items():
        print(f"  {comp_name:30s}: {comp_params:>12,}")

print(f"\n✅ Model initialized successfully!")
print(f"   Image Backbone: {IMAGE_CONFIG['backbone']}")
print(f"   Freeze Ratio: {IMAGE_CONFIG['freeze_ratio']}")
print(f"   Tabular Transformer: d_model={TRANSFORMER_CONFIG['d_model']}, heads={TRANSFORMER_CONFIG['nhead']}, layers={TRANSFORMER_CONFIG['num_layers']}")
print(f"   Cross-Attention Heads: {ATTENTION_CONFIG['attention_heads']}")

## 5. Train Model

In [ ]:
# Train the model
trained_model, history, best_metrics = train_attention_model(
    model=model,
    dataloaders=dataloaders,
    stats=stats,
    config=config_dict,
    device=DEVICE
)

print("\n" + "="*80)
print("🎉 TRAINING COMPLETED SUCCESSFULLY!")
print("="*80)

## 6. Visualize Training History

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0, 0].plot(history['train_loss'], label='Train', linewidth=2)
axes[0, 0].plot(history['val_loss'], label='Validation', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# RMSE
axes[0, 1].plot(history['train_rmse'], label='Train', linewidth=2)
axes[0, 1].plot(history['val_rmse'], label='Validation', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('RMSE ($)', fontsize=12)
axes[0, 1].set_title('Training and Validation RMSE', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# R²
axes[1, 0].plot(history['train_r2'], label='Train', linewidth=2)
axes[1, 0].plot(history['val_r2'], label='Validation', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('R² Score', fontsize=12)
axes[1, 0].set_title('Training and Validation R²', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Learning Rate
axes[1, 1].plot(history['learning_rate'], linewidth=2, color='green')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Training history plot saved to: ../reports/training_history.png")

## 7. [NEW] Attention Visualizations on Validation Set

Generate attention heatmaps to understand what the model focuses on.

In [ ]:
print("\n🎨 Generating Attention Visualizations...")

# Load best model
checkpoint = load_checkpoint()
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create visualizer
visualizer = AttentionVisualizer(model, DEVICE)

# Generate visualizations for validation samples
viz_dir = VISUALIZATION_CONFIG['save_dir']
viz_dir.mkdir(parents=True, exist_ok=True)

visualizations = visualizer.visualize_batch(
    dataloaders['val'],
    num_samples=VISUALIZATION_CONFIG['num_samples'],
    save_dir=viz_dir
)

print(f"\n✅ Attention visualizations saved to: {viz_dir}")
print(f"   Open these images to see what the model attends to!")

## 8. [NEW] Analyze Attention Patterns

Compute statistics about attention behavior across the validation set.

In [ ]:
print("\n📊 Analyzing Attention Patterns Across Validation Set...")

attention_analysis = analyze_attention_patterns(
    model, dataloaders['val'], DEVICE, stats,
    num_samples=VISUALIZATION_CONFIG['num_analysis_samples']
)

print(f"\n📈 Attention Pattern Analysis:")
print(f"  Mean Entropy: {attention_analysis['mean_entropy']:.4f} ± {attention_analysis['std_entropy']:.4f}")
print(f"  Mean Max Attention: {attention_analysis['mean_max_attention']:.4f}")
print(f"\n  Head-wise Mean Activations:")
for i, activation in enumerate(attention_analysis['head_activations_mean']):
    print(f"    Head {i+1}: {activation:.4f}")

# Plot head activation distribution
plt.figure(figsize=(10, 6))
heads = np.arange(len(attention_analysis['head_activations_mean']))
plt.bar(heads, attention_analysis['head_activations_mean'], 
        yerr=attention_analysis['head_activations_std'],
        alpha=0.7, capsize=5)
plt.xlabel('Attention Head', fontsize=12)
plt.ylabel('Mean Activation', fontsize=12)
plt.title('Attention Head Activation Distribution', fontsize=14, fontweight='bold')
plt.xticks(heads, [f'H{i+1}' for i in heads])
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(viz_dir / 'attention_head_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Attention analysis plot saved to: {viz_dir / 'attention_head_distribution.png'}")

## 9. Summary

**Enhanced Model with:**
1. ✅ Tabular Transformer replacing MLP
2. ✅ Cross-modal attention mechanism
3. ✅ Attention visualizations saved
4. ✅ Attention pattern analysis completed

**Next Steps:**
- Review attention visualizations in `reports/attention_visualizations/`
- Analyze which image regions the model focuses on
- Compare results with Approach 1
- Consider Approach 3 based on learnings

In [ ]:
print("\n" + "="*80)
print("🎉 NOTEBOOK EXECUTION COMPLETE!")
print("="*80)
print(f"\n📊 Best Validation Metrics:")
print(f"  RMSE: ${best_metrics['rmse']:>12,.2f}")
print(f"  R²:   {best_metrics['r2']:>8.4f}")
print(f"  MAE:  ${best_metrics['mae']:>12,.2f}")
print(f"\n💾 Model saved to: {MODELS_DIR / 'best_attention_model.pth'}")
print(f"🎨 Visualizations saved to: {viz_dir}")
print(f"\n👀 Check the visualizations to see what the model learned to attend to!")